<a href="https://colab.research.google.com/github/Aaricis/Introduction-to-Generative-AI-2024-Spring/blob/main/FineTune/Stable_Diffusion_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenAI HW10: Stable Diffusion Fine-tuning
In this homework, you will fine-tune your own Stable Diffusion model to generate your customized images from given text description. For more details, please refer to homework slides

## **TODOs**

1. Read the slides and make sure you know the objectives of this homework.
2. Save a copy of this Colab notebook.
3. Follow the steps in this Colab notebook to fine-tune your Stable Diffusion.
4. Evaluate outputs using FaceNet and CLIP
5. Update results and datasets to NTU COOL
If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to ntu-gen-ai-2024-spring-ta@googlegroups.com

Tips: At the top of each cell, it shows whether you need to change hyperparameters in that cell and how long it might take.

This is based on the work of [Hugging Face](https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image_lora.py).

And special thanks to [Celebrity Face Image Dataset](https://www.kaggle.com/datasets/vishesh1412/celebrity-face-image-dataset).


Thank you!


In [ ]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
    from google.colab.output import clear as clear_output
else:
    from IPython.display import clear_output


#@markdown ## Link to Google Drive
#@markdown This cell will load some requirements and create the necessary folders in your Google Drive. <p>
#@markdown Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset.
project_name = "Brad" #@param {type:"string"}
project_name = project_name.strip()
# dataset_name = "Brad-512" #@param ["Brad-512", "Anne-512"]
dataset_name = "Brad"

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
    print("Please write a valid project_name.")
else:
    if COLAB and not os.path.exists('/content/drive'):
      from google.colab import drive
      print("📂 Connecting to Google Drive...")
      drive.mount('/content/drive')

    project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
    project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

    root_dir = "/content" if COLAB else "~/Loras"
    main_dir        = os.path.join(root_dir, "drive/MyDrive/GenAI-HW10") if COLAB else root_dir
    project_dir =  os.path.join(main_dir, project_name)
    os.makedirs(main_dir, exist_ok=True)
    os.makedirs(project_dir, exist_ok=True)
    zip_file = os.path.join(main_dir, "Datasets.zip")
    !gdown 1OXPG2vNb8bG2334HML8vKpqo8UbAEV3d -O {zip_file}
    !unzip -q -o {zip_file} -d {main_dir}
    log_file = os.path.join(project_dir, "logs.zip")
    log_dir = os.path.join(project_dir, "logs")
    !git clone https://huggingface.co/yahcreeper/GenAI-HW10-Model {log_dir}
    # !cd {log_dir}
    # !git lfs pull
    # !gdown 1kalT3k7kEV0xcD6pf_OTSo7npHmsfZ_z -O {log_file}
    # !unzip -q -o {log_file} -d {project_dir}
    # !rm -f {log_file}
    model_path = os.path.join(project_dir, "logs", "checkpoint-last")
    images_folder   = os.path.join(main_dir, "Datasets", dataset_name)
    prompts_folder  = os.path.join(main_dir, "Datasets", "prompts")
    captions_folder = images_folder
    os.makedirs(images_folder, exist_ok=True)

    print(f"✅ Project {project_name} is ready!")
    step1_installed_flag = True

📂 Connecting to Google Drive...
Mounted at /content/drive
Downloading...
From: https://drive.google.com/uc?id=1OXPG2vNb8bG2334HML8vKpqo8UbAEV3d
To: /content/drive/MyDrive/GenAI-HW10/Datasets.zip
100% 2.78M/2.78M [00:00<00:00, 24.3MB/s]
fatal: destination path '/content/drive/MyDrive/GenAI-HW10/Brad/logs' already exists and is not an empty directory.
✅ Project Brad is ready!


In [ ]:
#@markdown ##  Install the required packages
#@markdown In this session, we will install some well-established packages to facilitate the fine-tuning process. <p>
#@markdown The installation will take about 5 minutes.
os.chdir(root_dir)
!pip -q install timm==1.0.7
!pip -q install fairscale==0.4.13
!pip -q install transformers==4.41.2
!pip -q install requests==2.32.3
!pip -q install accelerate==0.31.0
!pip -q install diffusers==0.29.1
!pip -q install einop==0.0.1
!pip -q install safetensors==0.4.3
!pip -q install voluptuous==0.15.1
!pip -q install jax==0.4.33
!pip -q install peft==0.11.1
!pip -q install deepface==0.0.92
!pip -q install tensorflow==2.17.0
!pip -q install keras==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ..

In [ ]:
#@markdown ##  Import necessary packages
#@markdown It is recommmended NOT to change codes in this cell.
import argparse
import logging
import math
import os
import random
import glob
import shutil
from pathlib import Path
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers

# Python Imaging Library（PIL）图像处理
from PIL import Image

# 图像处理
from torchvision import transforms
from torchvision.utils import save_image

# 显示进度条
from tqdm.auto import tqdm

# Parameter-Efficient Fine-tuning(PEFT)库
from peft import LoraConfig, get_peft_model
from peft.utils import get_peft_model_state_dict

# Hugging Face transformers
from transformers import AutoProcessor, AutoModel, CLIPTextModel, CLIPTokenizer

# Hugging Face Diffusion Model库
import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.utils import convert_state_dict_to_diffusers
from diffusers.training_utils import compute_snr
from diffusers.utils.torch_utils import is_compiled_module
from diffusers.models.modeling_outputs import Transformer2DModelOutput
# 面部检测
from deepface import DeepFace

# OpenCV
import cv2

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


24-11-17 09:35:44 - Directory /root/.deepface created
24-11-17 09:35:44 - Directory /root/.deepface/weights created


In [ ]:
# Do not change the following parameters, or the process may crashed due to GPU out of memory.
output_folder = os.path.join(project_dir, "logs") # 存放model checkpoints跟validation結果的資料夾
seed = 1126 # random seed
train_batch_size = 2 # training batch size
resolution = 512 # Image size
weight_dtype = torch.bfloat16 # 模型权重的数据类型是bfloat16
snr_gamma = 5 # Signal-to-Noise Ratio(SNR)信噪比缩放因子
#####

#@markdown ## Important parameters for fine-tuning Stable Diffusion
pretrained_model_name_or_path = "stablediffusionapi/cyberrealistic-41"
lora_rank = 32 # r(rank)
lora_alpha = 16 # Lora的alpha参数
#@markdown ### ▶️ Learning Rate
#@markdown The learning rate is the most important for your results. If you want to train slower with lots of images, or if your dim and alpha are high, move the unet to 2e-4 or lower. <p>
#@markdown The text encoder helps your Lora learn concepts slightly better. It is recommended to make it half or a fifth of the unet. If you're training a style you can even set it to 0.
learning_rate = 1e-4 #@param {type:"number"}
unet_learning_rate = learning_rate
text_encoder_learning_rate = learning_rate
lr_scheduler_name = "cosine_with_restarts" # 使用Cosine Annealing with Restarts学习率调度方法
lr_warmup_steps = 100 # 初始预热步数
#@markdown ### ▶️ Steps
#@markdown Choose your training step and the number of generated images per each validaion
max_train_steps = 1000 #@param {type:"slider", min:200, max:2000, step:100}

# 验证集数据
validation_prompt = "validation_prompt.txt"
validation_prompt_path = os.path.join(prompts_folder, validation_prompt)
validation_prompt_num = 3 #@param {type:"slider", min:1, max:5, step:1}
validation_step_ratio = 1 #@param {type:"slider", min:0, max:1, step:0.1}
with open(validation_prompt_path, "r") as f:
    validation_prompt = [line.strip() for line in f.readlines()]
#####

# LoRA Config

Stable Diffusion模型包含三个组件：CLIP、U-net、VAE。参数量分布和占比为：
[来源](https://forums.fast.ai/t/stable-diffusion-parameter-budget-allocation/101515)

| 组件      | 参数量 | 文件大小  |  占比 |
| ----------- | ----------- |----------- |----------- |
| CLIP      | 123,060,480 |  492 MB  |  12%  |
|  VAE  | 83,653,863 |  335 MB  |  8%  |
| U-net      | 859,520,964 |  3.44 GB  |  80%  |
| Total      | 1,066,235,307 | 4.27 GB |  100%  |

U-net是最核心的组件，CLIP相对也比较重要。因此，我们选择U-net和CLIP的Attention模块进行微调。




In [ ]:
# 加载模型
pipeline = DiffusionPipeline.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        safety_checker=None,
    )

# 查看模型的主要模块
print("UNet model:", pipeline.unet)
# print("VAE model:", pipeline.vae)
print("Text Encoder(CLIP):", pipeline.text_encoder)
# print("Tokenizer:", pipeline.tokenizer)


text_encoder/model.safetensors not found


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelin

UNet model: UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features

使用LoRA微调各组件的Attention模块

|组件|target_modules|
|------|------|
|CLIP|to_q,to_k,to_v,to_out|
|U-net|k_proj,v_proj,q_proj,out_proj|

In [ ]:
# Stable Diffusion LoRA设置
lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    target_modules=[
        "q_proj", "v_proj", "k_proj", "out_proj",  # 指定Text encoder(CLIP)的LoRA应用对象（用于调整注意力机制中的投影矩阵）
        "to_k", "to_q", "to_v", "to_out.0"  # 指定UNet的LoRA应用对象（用于调整UNet中的注意力机制）
    ],
    lora_dropout=0
)

## Define Some Useful Functions and Class

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_EXTENSIONS = [".png", ".jpg", ".jpeg", ".webp", ".bmp", ".PNG", ".JPG", ".JPEG", ".WEBP", ".BMP"]
train_transform = transforms.Compose(
    [
        transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(resolution),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)
class Text2ImageDataset(torch.utils.data.Dataset):
    """
    (1) Goal:
        - This class is used to build dataset for finetuning text-to-image model

    """
    def __init__(self, images_folder, captions_folder, transform, tokenizer):
        """
        (2) Arguments:
            - images_folder: str, path to images
            - captions_folder: str, path to captions
            - transform: function, turn raw image into torch.tensor
            - tokenizer: CLIPTokenize, turn sentences into word ids
        """
        self.image_paths = []
        for ext in IMAGE_EXTENSIONS:
            self.image_paths.extend(glob.glob(f"{images_folder}/*{ext}"))
        self.image_paths = sorted(self.image_paths)

        # 遍历图像路径，使用DeepFace提取面部特征嵌入
        self.train_emb = torch.tensor([DeepFace.represent(img_path, detector_backend="ssd", model_name="GhostFaceNet", enforce_detection=False)[0]['embedding'] for img_path in self.image_paths])
        caption_paths = sorted(glob.glob(f"{captions_folder}/*txt"))
        captions = []
        for p in caption_paths:
            with open(p, "r") as f:
                captions.append(f.readline())
        # 将文本转化为token
        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        self.input_ids = inputs.input_ids
        self.transform = transform

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        input_id = self.input_ids[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            # convert to tensor temporarily so dataloader will accept it
            tensor = self.transform(image)
        except Exception as e:
            print(f"Could not load image path: {img_path}, error: {e}")
            return None


        return tensor, input_id

    def __len__(self):
        return len(self.image_paths)


def prepare_lora_model(pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5", model_path=None):
    """
    (1) Goal:
        - This function is used to get the whole stable diffusion model with lora layers and freeze non-lora parameters, including Tokenizer, Noise Scheduler, UNet, Text Encoder, and VAE

    (2) Arguments:
        - pretrained_model_name_or_path: str, model name from Hugging Face
        - model_path: str, path to pretrained model.

    (3) Returns:
        - output: Tokenizer, Noise Scheduler, UNet, Text Encoder, and VAE

    """
    noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")

    # CLIP模型的分词器，用于将文本字符串转换为token id序列
    tokenizer = CLIPTokenizer.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="tokenizer"
    )

    # CLIPTextModel是CLIP模型的文本编码器部分，用于将文本转换为嵌入表示
    text_encoder = CLIPTextModel.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        subfolder="text_encoder"
    )

    # AutoencoderKL是一个VAE模型，用于将图像编码到潜在空间
    vae = AutoencoderKL.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="vae"
    )

    # UNet2DConditionModel是一个用于扩散模型的U-Net模型，用于在生成过程中预测噪声
    unet = UNet2DConditionModel.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        subfolder="unet"
    )

    # 将LoRA集成到text_encoder和unet
    text_encoder = get_peft_model(text_encoder, lora_config)
    unet = get_peft_model(unet, lora_config)

    # 打印可训练参数
    text_encoder.print_trainable_parameters()
    unet.print_trainable_parameters()


    # text_encoder = torch.load(os.path.join(model_path, "text_encoder.pt"))
    # unet = torch.load(os.path.join(model_path, "unet.pt"))

    # 冻结vae参数

    vae.requires_grad_(False)

    unet.to(DEVICE, dtype=weight_dtype)
    vae.to(DEVICE, dtype=weight_dtype)
    text_encoder.to(DEVICE, dtype=weight_dtype)
    return tokenizer, noise_scheduler, unet, vae, text_encoder

def prepare_optimizer(unet, text_encoder, unet_learning_rate=5e-4, text_encoder_learning_rate=1e-4):
    """
    (1) Goal:
        - This function is used to feed trainable parameters from UNet and Text Encoder in to optimizer each with different learning rate

    (2) Arguments:
        - unet: UNet2DConditionModel, UNet from Hugging Face
        - text_encoder: CLIPTextModel, Text Encoder from Hugging Face
        - unet_learning_rate: float, learning rate for UNet
        - text_encoder_learning_rate: float, learning rate for Text Encoder

    (3) Returns:
        - output: Optimizer

    """
    # 筛选UNet模型中需要梯度的参数
    unet_lora_layers = list(filter(lambda p: p.requires_grad, unet.parameters()))
    # 筛选text_encoder中需要梯度的参数
    text_encoder_lora_layers = list(filter(lambda p: p.requires_grad, text_encoder.parameters()))

    # 配置可训练参数列表
    trainable_params = [
        {"params": unet_lora_layers, "lr": unet_learning_rate},
        {"params": text_encoder_lora_layers, "lr": text_encoder_learning_rate}
    ]

    # 创建优化器
    optimizer = torch.optim.AdamW(
        trainable_params,
        lr=unet_learning_rate,
    )
    return optimizer

def evaluate(pretrained_model_name_or_path, weight_dtype, seed, unet_path, text_encoder_path, validation_prompt, output_folder, train_emb):
    """
    (1) Goal:
        - This function is used to evaluate Stable Diffusion by loading UNet and Text Encoder from the given path and calculating face similarity, CLIP score, and the number of faceless images.

    (2) Arguments:
        - pretrained_model_name_or_path: str, model name from Hugging Face
        - weight_dtype: torch.type, model weight type
        - seed: int, random seed
        - unet_path: str, path to UNet model checkpoint
        - text_encoder_path: str, path to Text Encoder model checkpoint
        - validation_prompt: list, list of str storing texts for validation
        - output_folder: str, directory for saving generated images
        - train_emb: tensor, face features of training images

    (3) Returns:
        - output: face similarity, CLIP score, the number of faceless images

    """
    pipeline = DiffusionPipeline.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        safety_checker=None,
    )
    pipeline.unet = torch.load(unet_path)
    pipeline.text_encoder = torch.load(text_encoder_path)
    pipeline = pipeline.to(DEVICE)
    clip_model_name = "openai/clip-vit-base-patch32"
    clip_model = AutoModel.from_pretrained(clip_model_name)
    clip_processor = AutoProcessor.from_pretrained(clip_model_name)

    # run inference
    with torch.no_grad():
        generator = torch.Generator(device=DEVICE) # 创建一个新的伪随机数生成器
        generator = generator.manual_seed(seed) # 设置随机数种子
        face_score = 0
        clip_score = 0
        mis = 0
        print("Generating validaion pictures ......")
        images = []
        for i in range(0, len(validation_prompt), 4): # 遍历validation_prompt，每次处理4个提示
            # 使用pipeline根据提示生成图像，添加到images列表中
            images.extend(pipeline(validation_prompt[i:min(i + 4, len(validation_prompt))], num_inference_steps=30, generator=generator).images)

        # 计算面部相似度和CLIP分数
        print("Calculating validaion score ......")
        valid_emb = []
        for i, image in enumerate(tqdm(images)):
            torch.cuda.empty_cache()
            save_file = f"{output_folder}/valid_image_{i}.png"
            image.save(save_file) # 将生成的图像保存到指定目录
            opencvImage = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR) # 使用OpenCV的cv2Color函数将图像从RGB颜色空间转化为BGR颜色空间
            emb = DeepFace.represent( # 使用DeepFace库提取面部特征嵌入
                opencvImage,
                detector_backend="ssd",
                model_name="GhostFaceNet",
                enforce_detection=False,
            )
            if emb == [] or emb[0]['face_confidence'] == 0: # 统计无法检测到面部的图片
                mis += 1
                continue
            # 计算CLIP分数，评估图片与文本的匹配程度
            emb = emb[0]
            inputs = clip_processor(text=validation_prompt[i], images=image, return_tensors="pt") # 处理CLIP模型的输入，输入文本和图像，返回适合模型输入的格式
            with torch.no_grad():
                outputs = clip_model(**inputs) # 使用CLIP模型，计算文本和图像的相似度
            sim = outputs.logits_per_image # 提取相似度分数
            clip_score += sim.item() # tensor转为数值
            valid_emb.append(emb['embedding'])
        if len(valid_emb) == 0:
            return 0, 0, mis
        valid_emb = torch.tensor(valid_emb)
        valid_emb = (valid_emb / torch.norm(valid_emb, p=2, dim=-1)[:, None]).cuda() # 归一化处理
        train_emb = (train_emb / torch.norm(train_emb, p=2, dim=-1)[:, None]).cuda()
        face_score = torch.cdist(valid_emb, train_emb, p=2).mean().item() # 计算距离
        # face_score = torch.min(face_score, 1)[0].mean()
        clip_score /= len(validation_prompt) - mis # 计算平均相似度
    return face_score, clip_score, mis

## Prepare Dataset, LoRA model, and Optimizer
Declare everything needed for Stable Diffusion fine-tuning.

In [ ]:
# 加载带LoRA层的Stable Diffusion模型
tokenizer, noise_scheduler, unet, vae, text_encoder = prepare_lora_model(pretrained_model_name_or_path, model_path)


optimizer = prepare_optimizer(
    unet,
    text_encoder,
    unet_learning_rate,
    text_encoder_learning_rate
)

lr_scheduler = get_scheduler(
    lr_scheduler_name,
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps,
    num_training_steps=max_train_steps,
    num_cycles=3
)

dataset = Text2ImageDataset(
    images_folder=images_folder,
    captions_folder=captions_folder,
    transform=train_transform,
    tokenizer=tokenizer,
)

# 自定义批处理函数，将多个样本（examples）组合成一个batch
def collate_fn(examples):
    pixel_values = []
    input_ids = []
    for tensor, input_id in examples:
        pixel_values.append(tensor)
        input_ids.append(input_id)

    # 图像tensor堆叠成一个多维tensor
    pixel_values = torch.stack(pixel_values, dim=0).float()
    # input_ids堆叠成一个多维tensor
    input_ids = torch.stack(input_ids, dim=0)
    return {"pixel_values": pixel_values, "input_ids": input_ids}

# 使用pytorch DataLoader加载数据集
train_dataloader = torch.utils.data.DataLoader(
    dataset,
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=train_batch_size,
    num_workers=8,
)
print("Preparation Finished!")

scheduler/scheduler_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

24-11-17 09:41:07 - Pre-trained weights is just downloaded to /root/.deepface/weights/ghostfacenet_v1.h5
24-11-17 09:41:07 - deploy.prototxt will be downloaded...


Downloading...
From: https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
To: /root/.deepface/weights/deploy.prototxt
28.1kB [00:00, 28.0MB/s]                   


24-11-17 09:41:08 - res10_300x300_ssd_iter_140000.caffemodel will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
To: /root/.deepface/weights/res10_300x300_ssd_iter_140000.caffemodel
100%|██████████| 10.7M/10.7M [00:00<00:00, 175MB/s]


Preparation Finished!


## Start Fine-tuning
This cell takes 25 minutes to run in the default setting, but it may vary depending on the condition of Colab and `max_train_step`.

In [ ]:
# 禁用tokenizer的并行化处理，在使用Hugging Face Transformers库的tokenizer时，经常用到的设置
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# PyTorch中用于控制Memory Efficient Softmax with DP (SDP)，禁用内存高效的softmax计算（关闭内存高效SDP）
# torch.backends.cuda.enable_mem_efficient_sdp(False)

# 禁用Flash Attention和Softmax with DP
# torch.backends.cuda.enable_flash_sdp(False)

# 进度条
progress_bar = tqdm(
    range(0, max_train_steps),
    initial=0,
    desc="Steps",
)

global_step = 0
num_epochs = math.ceil(max_train_steps / len(train_dataloader))
validation_step = int(max_train_steps * validation_step_ratio)
best_face_score = float("inf")
for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    unet.train()
    text_encoder.train()
    for step, batch in enumerate(train_dataloader):
        if global_step >= max_train_steps:
            break

        # 使用vae将图像编码为latent representation
        latents = vae.encode(batch["pixel_values"].to(DEVICE, dtype=weight_dtype)).latent_dist.sample()
        latents = latents * vae.config.scaling_factor

        # Sample noise that we'll add to the latents
        noise = torch.randn_like(latents)
        bsz = latents.shape[0]
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
        timesteps = timesteps.long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Get the text embedding for conditioning
        encoder_hidden_states = text_encoder(batch["input_ids"].to(latents.device), return_dict=False)[0]
        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(latents, noise, timesteps)

        # 输入噪声、时间步、text embedding，使用unet进行预测
        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
        if not snr_gamma: # 不使用snr_gamma
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean") # 标准均方误差损失函数
        else: # 使用snr_gamma，基于信噪比对损失进行加权
            snr = compute_snr(noise_scheduler, timesteps) #计算给定timestep的snr值
            mse_loss_weights = torch.stack([snr, snr_gamma * torch.ones_like(timesteps)], dim=1).min(
                dim=1
            )[0] #使用snr和snr_gamma计算每个timestep的损失权重
            if noise_scheduler.config.prediction_type == "epsilon": # 噪声预测
                mse_loss_weights = mse_loss_weights / snr # 降低权重随信噪比的变化
            elif noise_scheduler.config.prediction_type == "v_prediction": # 速度预测
                mse_loss_weights = mse_loss_weights / (snr + 1) # # 进一步平滑调整权重

            loss = F.mse_loss(model_pred.float(), target.float(), reduction="none") # 计算逐元素的均方误差
            loss = loss.mean(dim=list(range(1, len(loss.shape)))) * mse_loss_weights # 对非批次维度取均值，得到每个样本的损失，乘以对应时间步的加权因子
            loss = loss.mean() # 对所有样本的加权损失求平均值，作为最终损失值

        # 反向传播
        loss.backward() # 计算梯度
        optimizer.step() # 更新模型参数
        lr_scheduler.step() # 更新学习率
        optimizer.zero_grad() # 梯度清零
        progress_bar.update(1) # 更新进度条
        global_step += 1 # 更新全局步数

        # 验证模型性能
        if global_step % validation_step == 0 or global_step == max_train_steps:
            # 保存当前检查点
            save_path = os.path.join(output_folder, f"checkpoint-last")
            unet_path = os.path.join(save_path, "unet.pt")
            text_encoder_path = os.path.join(save_path, "text_encoder.pt")
            print(f"Saving Checkpoint to {save_path} ......")
            os.makedirs(save_path, exist_ok=True)
            torch.save(unet, unet_path)
            torch.save(text_encoder, text_encoder_path)
            save_path = os.path.join(output_folder, f"checkpoint-{global_step + 1000}")
            os.makedirs(save_path, exist_ok=True)

            # 评估模型性能
            face_score, clip_score, mis = evaluate(
                pretrained_model_name_or_path=pretrained_model_name_or_path,
                weight_dtype=weight_dtype,
                seed=seed,
                unet_path=unet_path,
                text_encoder_path=text_encoder_path,
                validation_prompt=validation_prompt[:validation_prompt_num],
                output_folder=save_path,
                train_emb=dataset.train_emb
            )
            print("Step:", global_step, "Face Similarity Score:", face_score, "CLIP Score:", clip_score, "Faceless Images:", mis)
            if face_score < best_face_score: # 保存当前最佳结果
                best_face_score = face_score
                save_path = os.path.join(output_folder, f"checkpoint-best")
                unet_path = os.path.join(save_path, "unet.pt")
                text_encoder_path = os.path.join(save_path, "text_encoder.pt")
                os.makedirs(save_path, exist_ok=True)
                torch.save(unet, unet_path)
                torch.save(text_encoder, text_encoder_path)
print("Fine-tuning Finished!!!")

Steps:   0%|          | 0/400 [00:00<?, ?it/s]

Saving Checkpoint to /content/drive/MyDrive/GenAI-HW10/Brad/logs/checkpoint-last ......


model_index.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

text_encoder/model.safetensors not found


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

Generating validaion pictures ......


  0%|          | 0/30 [00:00<?, ?it/s]

Calculating validaion score ......


  0%|          | 0/3 [00:00<?, ?it/s]

Step: 400 Face Similarity Score: 1.2510902881622314 CLIP Score: 30.896265665690105 Faceless Images: 0
Fine-tuning Finished!!!


## Testing
The fine-tuning process is done. We then want to test our model.

We will first load the fine-tuned model for checkpoint we saved and calculate the face similarity, CLIP score, and the number of faceless images.
This process will take about 15 minutes.

In [ ]:
torch.cuda.empty_cache()
checkpoint_path = os.path.join(output_folder, f"checkpoint-last") # 設定使用哪個checkpoint inference
unet_path = os.path.join(checkpoint_path, "unet.pt")
text_encoder_path = os.path.join(checkpoint_path, "text_encoder.pt")
inference_path = os.path.join(project_dir, "inference")
os.makedirs(inference_path, exist_ok=True)
train_image_paths = []
for ext in IMAGE_EXTENSIONS:
    train_image_paths.extend(glob.glob(f"{images_folder}/*{ext}"))
train_image_paths = sorted(train_image_paths)
train_emb = torch.tensor([DeepFace.represent(img_path, detector_backend="ssd", model_name="GhostFaceNet", enforce_detection=False)[0]['embedding'] for img_path in train_image_paths])

face_score, clip_score, mis = evaluate(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    weight_dtype=weight_dtype,
    seed=seed,
    unet_path=unet_path,
    text_encoder_path=text_encoder_path,
    validation_prompt=validation_prompt,
    output_folder=inference_path,
    train_emb=train_emb,
)
print("Face Similarity Score:", face_score, "CLIP Score:", clip_score, "Faceless Images:", mis)

24-11-11 10:36:03 - Pre-trained weights is downloaded from https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5 to /root/.deepface/weights/ghostfacenet_v1.h5


Downloading...
From: https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5
To: /root/.deepface/weights/ghostfacenet_v1.h5
100%|██████████| 17.3M/17.3M [00:00<00:00, 84.6MB/s]


24-11-11 10:36:04 - Pre-trained weights is just downloaded to /root/.deepface/weights/ghostfacenet_v1.h5
24-11-11 10:36:04 - deploy.prototxt will be downloaded...


Downloading...
From: https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
To: /root/.deepface/weights/deploy.prototxt
28.1kB [00:00, 35.6MB/s]                   


24-11-11 10:36:05 - res10_300x300_ssd_iter_140000.caffemodel will be downloaded...


...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Generating validaion pictures ......


  0%|          | 0/30 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB. GPU 0 has a total capacity of 14.75 GiB of which 3.11 GiB is free. Process 10798 has 11.63 GiB memory in use. Of the allocated memory 11.34 GiB is allocated by PyTorch, and 28.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)